In [7]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [16]:
srcfld = Path("C:/git/py/tlak")
dstfld = Path("C:/git/py/tlak")

srcfile = "tlak.xlsx"
dstfile = "tlak_out.xlsx"

srcpath = srcfld / srcfile
dstpath = dstfld / dstfile

In [9]:
xl = pd.ExcelFile(srcpath)
df = xl.parse('mix', usecols = [0, 1, 2])
xl.close()
data_orig = df[df.columns[0:3]].to_numpy()

In [10]:
# Extract sys and dia columns from the data
sys_orig = data_orig[:, 0]
dia_orig = data_orig[:, 1]
pulse_orig = data_orig[:, 2]

# Prepare the feature matrix X with sys and dia values
X_orig = np.column_stack((sys_orig, dia_orig))

In [11]:
# Fit  linear regression models
model_sys_to_dia = LinearRegression()
model_sys_to_dia .fit(sys_orig.reshape(-1, 1), dia_orig)

model_dia_to_sys = LinearRegression()
model_dia_to_sys.fit(dia_orig.reshape(-1, 1), sys_orig)

model_dia_sys_to_pulse = LinearRegression()
model_dia_sys_to_pulse.fit(X_orig, pulse_orig)

LinearRegression()

In [12]:
dia_morning = [84, 88, 85, 86, 81, 87, 85, 93, 93, 87, 92, 87, 90, 84, 82, 82, 90, 87, 86, 82, 86, 90, 84, 104, 81, 95, 115, 98, 84, 88, 100, 98, 86, 88, 99, 90, 90, 89, 88, 98, 93, 94, 81, 97, 83, 82, 83, 81]
dia_evening = [89, 95, 89, 95, 91, 86, 94, 93, 98, 105, 81, 102, 82, 102, 87, 88, 87, 99, 89, 92, 89, 87, 103, 91, 89, 85, 91, 90, 93, 100, 98, 89, 96, 94, 94, 101, 96, 97, 100]

In [13]:
def get_sys_pulse (dia_orig, model_dia_to_sys ):
    
    # generate dia input values
    size = 50

    mean_dia_orig = np.mean(dia_orig)
    std_dia_orig = np.std(dia_orig)

    dia_values = np.random.normal(loc=mean_dia_orig, scale=std_dia_orig, size=size)
    dia_values = dia_values.astype(int)

    # Predict values 
    sys_predicted = model_dia_to_sys.predict(np.array(dia_values).reshape(-1, 1))
    sys_dia_values = np.column_stack((sys_predicted, dia_values))
    pulse_predicted = model_dia_sys_to_pulse.predict(sys_dia_values)

    sys_out = sys_predicted.astype(int) 
    dia_out = dia_values 
    pulse_out = pulse_predicted.astype(int)

    # sys_out, dia_out, pulse_out
    np_out = np.column_stack((sys_out, dia_out, pulse_out))
    print(np_out.mean(axis=0))

    return np_out



In [14]:
np_morning_out = get_sys_pulse (dia_morning , model_dia_to_sys )
np_evening_out = get_sys_pulse (dia_evening , model_dia_to_sys )

[131.    88.    68.86]
[134.18  92.84  71.2 ]


In [15]:
combined_array = np.hstack((np_morning_out, np_evening_out))

columns = ['sys_m', 'dia_m', 'pulse_m', 'sys_e', 'dia_e', 'pulse_e']
df_out = pd.DataFrame(combined_array, columns=columns)

with pd.ExcelWriter(dstpath) as writer:  
    df_out.to_excel(writer, sheet_name='Tlak_dia_to_sys_pulse')
    # df2.to_excel(writer, sheet_name='Sheet_name_2')